# Итоговый проект DS

* В качестве моделей для тестов выбраны LLama 3.2 и Mistral   
mistral:latest     f974a74358d6    4.1 GB    12 seconds ago  
llama3.2:latest    a80c4f17acd5    2.0 GB    3 weeks ago  
* Для каждой модели будут проведены экмперементы с 2 эмбеддерами MiniLM-L12-v2 и mpnet-base-v2
* База данных RubQ 2.0

In [102]:
#!pip install sentence-transformers

В качестве эмбедеров не подойдут такие модели наприммер как word2vec, так-как
* они не учитывают контекст и каждое слово всегда имеет одно значение 
* а также не умеют понимать предложение и текст полностью так как работают на уровне слов

Преимущества SentenceTransformer

* Хорошо учитвают контекст 
* Хорошо справляются с семантическим поиском

In [1]:
from sentence_transformers import SentenceTransformer

embedder_light = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
# этот эмбеддер показался мне куда хуже, так то временно он будет закомментирован  
#embedder_heavy = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

d:\pypy\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain_openai import ChatOpenAI

# Mistral 7B
model_mistral = ChatOpenAI(base_url="http://localhost:11434/v1", api_key='1', model="mistral")

# LLaMA 3.2
model_llama = ChatOpenAI(base_url="http://localhost:11434/v1", api_key='1', model="llama3.2")

In [3]:
import pandas as pd

splits = {'test': 'data/test-00000-of-00001-d519841742f463e6.parquet', 'dev': 'data/dev-00000-of-00001-d7e3040a344e1e68.parquet'}
df = pd.read_parquet("hf://datasets/d0rj/RuBQ_2.0/" + splits["test"])

In [4]:
df[['question_text', 'answer_text']].head()

,question_text,answer_text
0,Что может вызвать цунами?,Землетрясение
1,Кто написал роман «Хижина дяди Тома»?,Г. Бичер-Стоу
2,Кто автор пьесы «Ромео и Джульетта»?,Шекспир
3,Как называется столица Румынии?,Бухарест
4,На каком инструменте играл Джимми Хендрикс?,Гитара


In [5]:
from chromadb import Client, Settings


# Инициализация Chroma
# client = Client(settings=Settings(persist_directory="./chroma_db"))
# collection = client.create_collection(name="rubq")

# Создание эмбеддингов и добавление в базу
# for idx, row in df[['question_text', 'answer_text']].iterrows():
#     question = row['question_text']
#     answer = row['answer_text']
#     embedding = embedder_heavy.encode(question)  # используем тяжелый эмбедер
#     collection.add(ids=[str(idx)], embeddings=[embedding], documents=[answer])

In [7]:
# запускать если надо создать коллекцию
from chromadb import Client, Settings

# Инициализация Chroma
client = Client(settings=Settings(persist_directory="./chroma_db"))
# Создание новой коллекции для легкого эмбеддера
collection_light = client.create_collection(name="rubq_light")

for idx, row in df[['question_text', 'answer_text']].iterrows():
    question = row['question_text']
    answer = row['answer_text']
    embedding = embedder_light.encode(question)  # Используем легкий эмбеддер
    
    # Добавляем данные в коллекцию, включая метаданные (вопросы)
    collection_light.add(
        ids=[str(idx)], 
        embeddings=[embedding], 
        documents=[answer], 
        metadatas=[{"question": question}]  # Сохраняем вопрос как метаданные
    )


In [104]:
# если надо удалить
client.delete_collection(name="rubq_light")

In [8]:
# если уже создана
from chromadb import Client, Settings

client = Client(settings=Settings(persist_directory="./chroma_db"))
collection_light = client.get_collection(name="rubq_light")

In [9]:
def retrieve_from_db(collection, query, embedder, n_results=1):
    # Генерируем эмбеддинг для пользовательского запроса
    query_embedding = embedder.encode(query)
    
    # Выполняем запрос в ChromaDB
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results
    )
    
    # Получаем документы, метаданные (в данном случае это вопросы), и соответствующие идентификаторы
    documents = results.get('documents', [])  # Список ответов
    metadatas = results.get('metadatas', [])  # Список вопросов
    ids = results.get('ids', [])              # Список идентификаторов
    
    # Формируем список вопрос-ответ пар
    qa_pairs = [
        {"id": doc_id, "question": meta, "answer": doc}
        for doc_id, meta, doc in zip(ids, metadatas, documents)
    ]
    
    res = {}
    for item in qa_pairs:
        res["Вопрос:"] =  item['question'][0]['question']
        res["Ответ:"] = item['answer'][0]
    
    return res


In [8]:
# results = retrieve_from_db(collection, "Кто автор Хижина дяди Тома", embedder_heavy)
# for doc in results:
#     print(doc)

# тут вывод 
# совсем не соответсвет контексту 
# топ1 по посику в бд - Шут а нужно Г. Бичер-Стоу

In [139]:
query = "играл Джимми Хендрикс?"
results = retrieve_from_db(collection_light, query, embedder_light, n_results=1)

print(results)
    


{'Вопрос:': 'На каком инструменте играл Джимми Хендрикс?', 'Ответ:': 'Гитара'}


In [10]:
from langchain_community.tools import DuckDuckGoSearchRun

In [11]:
def search_duckduckgo(query): 
    tool = DuckDuckGoSearchRun(max_results=3)
    results = tool.invoke(query)
    return results

print(search_duckduckgo('Какое удобрение подойдёт кактусу?'))

Если кактусу в течение долгого времени не хватает воды, он «молчать» не будет и расскажет вам о жажде с помощью следующих изменений: А если создать кактусу все условия, ... Для этого подойдёт удобрение с высоким содержанием калия (например, для томатов) или специальное удобрение для кактусов. Подкармливайте кактус раз в ... Удобрение кактусов ... какое освещение подходит кактусам и как обеспечить его правильный баланс. 1.1. Естественное освещение ... что позволяет кактусу аккуратно взять столько воды, сколько ... Как ухаживать за кактусом: полив, пересадка и болезни Чем подкормить кактусы в домашних условиях Причины, по которым кактус не цветет Как подобрать почву Удобрение кактусов Чрезмерное удобрение. В природе кактусы предпочитают песчаные, бедные гумусом почвы, содержащие мало питательных веществ. Это не значит, что растение вообще может обходиться без какого ...


In [18]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, Any
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage


from dotenv import load_dotenv
_ = load_dotenv()

In [17]:
import re
# Регулярное выражение для парсинга действий
action_re = re.compile('^Action: (\w+): (.*)$')

# # Определение структуры состояния агента
# class AgentState(TypedDict):
#     messages: Annotated[list[Any], operator.add]
#     memory: list[str]  # Память для хранения прошлых сообщений
    
    
class AgentState(TypedDict):
    messages: Annotated[list[Any], operator.add]  # Сообщения (вопросы и ответы)
    memory: list[str]  # Память (история диалога)
    db_results: dict  # Результаты поиска в БД
    web_results: str  # Результаты поиска в интернете


In [80]:
class Agent:
    def __init__(self, model, collection, embedder, system=""):
        self.model = model
        self.collection = collection
        self.embedder = embedder
        self.system = system
        self.graph = self._build_graph()

    def _build_graph(self):
        graph = StateGraph(AgentState)

        # Добавляем узлы
        graph.add_node("refine_query", self.refine_query)  # Новый узел
        graph.add_node("search_db", self.search_db)
        graph.add_node("evaluate_relevance", self.evaluate_relevance)
        graph.add_node("search_web", self.search_web)
        graph.add_node("edit_web_response", self.edit_web_response)

        # Добавляем переходы
        graph.add_edge("refine_query", "search_db")  # После уточнения ищем в БД
        graph.add_edge("search_db", "evaluate_relevance")
        graph.add_conditional_edges(
            "evaluate_relevance",
            self.decide_next_step,
            {"end": END, "search_web": "search_web"}
        )
        graph.add_edge("search_web", "edit_web_response")
        graph.add_edge("edit_web_response", END)

        # Устанавливаем начальный узел
        graph.set_entry_point("refine_query")

        return graph.compile()
    
    def refine_query(self, state: AgentState):
        """
        Уточняет запрос пользователя на основе контекста.
        """
        user_query = state["messages"][-1]["content"]
        context = "\n".join(state["memory"])  # История диалога
        
        if 'По поиску в RuBQ Ответ:' in context:
            context = context.replace('По поиску в RuBQ Ответ:', '')
        elif 'По поиску в DuckDuckGo:' in context:
            context = context.replace('По поиску в DuckDuckGo:') 
        # print('----')
        # print('Что в истории до редактирования', context )
        # print('----')
        
        REFINE_QUERY_PROMPT = """
            Ты ассистент для уточнения запросов пользователя.
                ### Твоя задача ###
                На основе предоставленной истории и вопроса пользователя составь уточнённый запрос, который будет максимально полным и самодостаточным. Если запрос уже ясен и не требует контекста, оставь его без изменений. Если запрос относится к новой теме, просто перепиши его как есть.

                ### Примеры ###
                История: "Кактусам подойдут удобрения с высоким содержанием калия."
                Запрос: "А сколько раз поливать?"
                Твоя корректировка: "Сколько раз поливать кактусы?"

                История: "Лучший язык для начинающих программистов — Python."
                Запрос: "Что насчет Java?"
                Твоя корректировка: "Подходит ли Java для начинающих программистов?"

                История: 
                Запрос: "Какова площадь Луны?"
                Твоя корректировка: "Какова площадь Луны?"

                История: "Кактусам подойдут удобрения с высоким содержанием калия."
                Запрос: "Расскажи про животных."
                Твоя корректировка: "Расскажи про животных."

                ### История ###
                {context}

                ### Запрос пользователя ###
                {human_message}

                ### Твоя корректировка ###
                """

        # Формируем промпт для уточнения запроса
        prompt = REFINE_QUERY_PROMPT.format(
            context=context,
            human_message=user_query
        )

        # Запрашиваем уточнение у LLM
        response = self.model.invoke(prompt)
        refined_query = response.content.strip()

        # Сохраняем уточнённый запрос в состоянии
        print('Отредактированый вопрос:', refined_query)
        state["messages"][-1]["content"] = refined_query
        return state

    def search_db(self, state: AgentState):
        """
        Ищет в базе данных и сохраняет результаты в состоянии.
        """
        user_query = state["messages"][-1]["content"]
        db_results = retrieve_from_db(self.collection, user_query, self.embedder)
        state["db_results"] = db_results
        return state

    def evaluate_relevance(self, state: AgentState):
        """
        Оценивает релевантность ответа из БД с помощью LLM.
        """
        user_query = state["messages"][-1]["content"]
        db_question = state["db_results"]["Вопрос:"]
        db_answer = state["db_results"]["Ответ:"]

        RELEVANCE_PROMPT = """
                Ты ассистент сравнения вопросов по семантической близости!
                Можно ли ответить на эти 2 вопроса одинаково правильно(Одинаковы ли они по смыслу)?
                1. {user_query}
                2. {db_question}

                Верни только "да" или "нет" и ничего кроме этого!
                """
        # Формируем промпт для оценки релевантности
        prompt = RELEVANCE_PROMPT.format(
            user_query=user_query,
            db_question=db_question,
            db_answer=db_answer
        )

        # Запрашиваем оценку у LLM
        response = self.model.invoke(prompt)
        # print(f'Вопрос бд:{db_question}, Ответ бд:{db_answer}')
        # print(f"Оценка релевантности: {response.content}")  # Отладочный вывод

        # Если ответ релевантен, добавляем его в память
        if "да" in response.content.lower():
            # state["memory"].append(f"По поиску в RuBQ Ответ: {db_answer}")
            self.update_memory(state, user_query, f"По поиску в RuBQ Ответ: {db_answer}")
            state["next_step"] = "end"  # Завершаем выполнение
        else:
            state["next_step"] = "search_web"  # Переходим к поиску в интернете

        return state

    def search_web(self, state: AgentState):
        """
        Ищет в интернете с помощью DuckDuckGo.
        """
        user_query = state["messages"][-1]["content"]
        web_results = self.search_duckduckgo(user_query)
        state["web_results"] = web_results
        return state

    def edit_web_response(self, state: AgentState):
        """
        Редактирует ответ из интернета с помощью LLM.
        """
        user_query = state["messages"][-1]["content"]
        web_results = state["web_results"]

        # Формируем промпт для редактирования
        prompt = f"""
        Ты получил ответ из интернета на вопрос пользователя. Отредактируй его, чтобы он был понятным и удобным для чтения.

        Вопрос пользователя: {user_query}
        Ответ из интернета: {web_results}

        Отредактированный ответ:
        """

        # Запрашиваем редактирование у LLM
        response = self.model.invoke(prompt)

        # Добавляем отредактированный ответ в память
        # state["memory"].append(f"По поиску в DuckDuckGo: {response.content}")
        self.update_memory(state, user_query, f"По поиску в DuckDuckGo: {response.content}")
        return state

    def decide_next_step(self, state: AgentState):
        """
        Определяет следующий шаг на основе состояния.
        """
        return state.get("next_step", "search_web")

    def search_duckduckgo(self, query):
        """
        Ищет в интернете с помощью DuckDuckGo.
        """
        tool = DuckDuckGoSearchRun(max_results=1)
        return tool.invoke(query)

    def update_memory(self, state: AgentState, user_message, agent_message):
        """
        Обновляет память агента.
        """
        state["memory"].append(f"User: {user_message}")
        state["memory"].append(f"Agent: {agent_message}")

    def invoke(self, state: AgentState):
        """
        Запускает агента с начальным состоянием.
        """
        return self.graph.invoke(state)

In [217]:
# Агент с Mistral и легким эмбеддером
#agent_mistral = Agent(model_mistral, [tool], system=prompt, collection=collection_light, embedder=embedder_light)

# Агент с LLaMA 2 и легким эмбеддером
#agent_llama = Agent(model_llama, [tool], system=prompt,  collection=collection_light, embedder=embedder_light)

# Инициализация модели и коллекции
model = model_mistral
collection = collection_light
embedder = embedder_light  

# Создание агента
agent = Agent(model, collection, embedder)

# Инициализация состояния
state = {
    "messages": [{"content": "На чем играл Джимми Хендрикс?"}],
    "memory": [],
    "db_results": None,
    "web_results": None
}

# Запуск агента
result = agent.invoke(state)

# Вывод результата
for entry in result["memory"]:
    print(entry)
    
    
state = {
    "messages": [{"content": "Какое удобрение подойдёт для Кактуса?"}],
    "memory": [],
    "db_results": None,
    "web_results": None
}

# Запуск агента
result = agent.invoke(state)

# Вывод результата
for entry in result["memory"]:
    print(entry)   

Вопрос бд:На каком инструменте играл Джимми Хендрикс?, Ответ бд:Гитара
Оценка релевантности:  Да
По поиску в RuBQ Ответ: Гитара
Вопрос бд:Какой пост занимает Шойгу?, Ответ бд:Губернатор Московской области
Оценка релевантности:  Нет
По поиску в DuckDuckGo:  Удобрения для кактусов в домашних условиях: Чем подкармливать растение?

   Для здорового роста кактуса, можно применять следующие типы удобрений:
   1. Удобрение с высоким содержанием калия (например, для томатов) или специальное удобрение для кактусов.

   Подкармливайте кактус раз в 2-3 недели во время активного роста и вегетации. Следуя этой рекомендации поможет избежать нарушения равновесия солей в почве и переобогащения, что может приводить к отравлению кактуса.

   Какой грунт выбрать для кактуса?
   1. Полиферт предлагает комплексное минеральное водорастворимое удобрение с микроэлементами в хелатной форме с различным соотношением фосфора, калия и азота.
   2. Лучшая стратегия для кактуса - обильный полив и очень хороший дрена

In [81]:
# Инициализация модели и коллекции
model = model_mistral
collection = collection_light
embedder = embedder_light  

# Создание агента
agent = Agent(model, collection, embedder)


In [35]:
state = {
    "messages": [],
    "memory": [],
    "db_results": None,
    "web_results": None
}
state['messages'] = [{"content":'Кто губернатор Саратовской области?'}]

# Запуск агента
result = agent.invoke(state)

# Вывод результата
print(result["memory"][-1])

Вопрос бд:Какой пост занимает Назарбаев?, Ответ бд:глава Совета безопасности Казахстана, председателя правящей партии «Нур Отан
Оценка релевантности:  Нет
По поиску в DuckDuckGo:  В настоящее время должность Губернатора Саратовской области занимает Роман Викторович Бусаргин. Его вступление в должность состоялось 16 сентября 2022 года после победы на выборах, прошедших с 9 по 11 сентября того же года. Он получил поддержку 72,55% избирателей (716 974 голоса), став официально зарегистрированным губернатором. Церемония инаугурации прошла в Правительстве области.

        Следует отметить, что 21 сентября предыдущего года Валерий Радаев официально вступил в должность Губернатора Саратовской области после назначения Президентом РФ.

        Сегодня, 29 июля, Роман Бусаргин отметил свой 43-й день рождения.


### Тесты с памятью

In [85]:
# тут в выводах много лишней информации, чтобы посмотреть как работают разные ллм узлы.

# Инициализация состояния
state = {
    "messages": [],
    "memory": [],
    "db_results": None,
    "web_results": None
}

# Первый запрос
state['messages'] = [{"content": "В каком виде спорта разыгрывают кубок Стэнли?"}]
result = agent.invoke(state)

# Вывод результата первого запроса
print("Ответ бота:", result["memory"][-1][7:])

print('--------------------------')

# Второй запрос (с учётом памяти)
state['messages'] = [{"content": "А кубок Гагарина?"}]
result = agent.invoke(state)

# Вывод результата второго запроса
print("Ответ бота:", result["memory"][-1][7:])

print('--------------------------')

# Второй запрос (с учётом памяти)
state['messages'] = [{"content": "А в каком виде спорта разыгрывают женский кубок Убер??"}]
result = agent.invoke(state)

# Вывод результата второго запроса
print("Ответ бота:", result["memory"][-1][7:])

Отредактированый вопрос: Какой вид спорта проводится Кубком Стэнли?
Ответ бота: По поиску в RuBQ Ответ: Хоккей
--------------------------
Отредактированый вопрос: Кубок Гагарина - это какой вид спорта?
Ответ бота: По поиску в RuBQ Ответ: Хоккей с шайбой
--------------------------
Отредактированый вопрос: В каком типе спорта разыгрывается Женский Кубок Убера?
Ответ бота: По поиску в RuBQ Ответ: в бадминтоне


# Итог 

* Я попробовал 2 совершенно разных подхода с оценкой релевантоности используя расстояния между эмюедингами что было достаточно плохим результатом   
  
### 1 вариант содержал следующий prompt  
   
prompt = """Ты — интеллектуальный ассистент, специализирующийся на ответах на вопросы на основе базы знаний и поиска в интернете. Твоя задача — предоставлять точные, полезные и структурированные ответы на вопросы пользователя, используя следующие возможности:

1. **База знаний**:
   - У тебя есть доступ к базе знаний RuBQ 2.0, содержащей вопросы и ответы на русском языке.
   - Если вопрос пользователя релевантен базе знаний, используй информацию из неё для формирования ответа.

2. **Поиск в интернете**:
   - Если информация отсутствует в базе знаний, используй поиск в интернете (DuckDuckGo) для нахождения ответа.
   - Обязательно проверяй достоверность найденной информации.

3. **Память диалога**:
   - Ты должен запоминать контекст диалога с пользователем, чтобы отвечать на последующие вопросы с учётом предыдущих.
   - Используй память для уточнения вопросов и предоставления более точных ответов.

4. **Формат ответа**:
   - Ответ должен быть чётким, структурированным и понятным.
   - Если информация найдена в базе знаний, начни ответ с: "Согласно базе знаний: [ответ]".
   - Если информация найдена в интернете, начни ответ с: "Согласно результатам поиска: [ответ]".
   - Если ответ требует уточнения, задай уточняющий вопрос пользователю.

5. **Работа с инструментами**:
   - Для поиска в базе знаний используй инструмент `retrieve_from_db`.
   - Для поиска в интернете используй инструмент `DuckDuckGoSearchRun`.
   - Если инструмент возвращает ошибку или некорректные данные, сообщи об этом пользователю и попробуй другой подход.

6. **Язык и стиль**:
   - Отвечай на русском языке.
   - Используй вежливый и профессиональный тон.
   - Избегай избыточного использования технических терминов, если пользователь не запрашивает их явно.

7. **Обработка ошибок**:
   - Если ты не можешь найти ответ на вопрос, честно сообщи об этом пользователю и предложи альтернативные варианты (например, уточнить вопрос или поискать в других источниках).

8. **Примеры работы**:
   - Вопрос: "Что такое квантовая механика?"
     Ответ: "Согласно базе знаний: Квантовая механика — это раздел физики, изучающий поведение частиц на атомном и субатомном уровнях."
   - Вопрос: "Как ухаживать за алоэ?"
     Ответ: "Согласно результатам поиска: Алоэ требует умеренного полива, яркого света и хорошо дренированной почвы."

9. **Важные правила**:
   - Никогда не выдумывай информацию. Если ты не уверен в ответе, скажи об этом.
   - Всегда проверяй релевантность информации перед тем, как использовать её в ответе.
   - Следи за контекстом диалога, чтобы избежать повторений или противоречий."""


#### с таким вариантом и другим построением графа модель работала очень плохо, почти никигда не обращалась к бд а использовала тольео те данные на которых была обучена.

*  Куда лучшей идей было сделать оценивание релевантности также при помощи LLM
*  Итоговой модейлью выбрана Mistral 8b так как Llama3.2 3b Очень часто выдаёт плохой результат 
 напрмер при вопросе `"Какое удабрение подойдет кактусам"` вопрос `"Какое растение перерабатывают в текилу"` она считает одинаковыми
 Mistral с оценкой справляется куда лучше
* Также с задачей отредактировать результат из интернета LLama плохо справляется вставляя пустые символы или используя слова на англйском иногда даже просто пишет транслитом 
например `Для кактосов mojet podoiti udobrenie naprimer`  